In [1]:
import findspark
findspark.init()

import pyspark
import random

In [2]:
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.session import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import isnan, when, count, col

In [3]:
filename = "2004.csv"

In [4]:
def _init_spark():
    spark = SparkSession.builder.appName("Project").getOrCreate()
    sc = spark.sparkContext
    return spark, sc

In [5]:
spark, sc = _init_spark()

In [6]:
sqlContext = SQLContext(sc)

df = sqlContext.read.load(filename, 
                      format='com.databricks.spark.csv', 
                      header='true',
                      delimiter=',',
                      inferSchema='true')
df.cache()

DataFrame[Year: int, Month: int, DayofMonth: int, DayOfWeek: int, DepTime: string, CRSDepTime: int, ArrTime: string, CRSArrTime: int, UniqueCarrier: string, FlightNum: int, TailNum: string, ActualElapsedTime: string, CRSElapsedTime: int, AirTime: string, ArrDelay: string, DepDelay: string, Origin: string, Dest: string, Distance: int, TaxiIn: int, TaxiOut: int, Cancelled: int, CancellationCode: string, Diverted: int, CarrierDelay: int, WeatherDelay: int, NASDelay: int, SecurityDelay: int, LateAircraftDelay: int]

In [7]:
# removing as is stated in the task along with the 'Year'
col_to_drop = ['ArrTime', 'ActualElapsedTime', 'AirTime', 'TaxiIn', 'Diverted', 
               'CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay', 'Year']
df = df.drop(*col_to_drop)

In [8]:
# "CancelationCode" has too much "null" (98% of the data) we will remove it too. Others have no missing values except for "TailNum", that has only 127 values left.  
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+-----+----------+---------+-------+----------+----------+-------------+---------+-------+--------------+--------+--------+------+----+--------+-------+---------+----------------+
|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|CRSElapsedTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiOut|Cancelled|CancellationCode|
+-----+----------+---------+-------+----------+----------+-------------+---------+-------+--------------+--------+--------+------+----+--------+-------+---------+----------------+
|    0|         0|        0|      0|         0|         0|            0|        0|    127|             0|       0|       0|     0|   0|       0|      0|        0|         7001506|
+-----+----------+---------+-------+----------+----------+-------------+---------+-------+--------------+--------+--------+------+----+--------+-------+---------+----------------+



In [9]:
# deletion of the "CancelationCode" and droping rows that contain "TailNum", "UniqueCarrier" 
# is represented by several values so we will explore it later
df = df.drop('CancellationCode')
df = df.drop('TailNum')
## df = df.filter(df.TailNum.isNotNull() )

In [10]:
df.printSchema()

root
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- DepTime: string (nullable = true)
 |-- CRSDepTime: integer (nullable = true)
 |-- CRSArrTime: integer (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: integer (nullable = true)
 |-- CRSElapsedTime: integer (nullable = true)
 |-- ArrDelay: string (nullable = true)
 |-- DepDelay: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: integer (nullable = true)
 |-- TaxiOut: integer (nullable = true)
 |-- Cancelled: integer (nullable = true)



In [11]:
# "ArrDelay" and "DepDelay" have string type. We cast them to Integer
df = df.withColumn("ArrDelay", df["ArrDelay"].cast(IntegerType()))
df = df.withColumn("DepDelay", df["DepDelay"].cast(IntegerType()))
df.printSchema()

root
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- DepTime: string (nullable = true)
 |-- CRSDepTime: integer (nullable = true)
 |-- CRSArrTime: integer (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: integer (nullable = true)
 |-- CRSElapsedTime: integer (nullable = true)
 |-- ArrDelay: integer (nullable = true)
 |-- DepDelay: integer (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: integer (nullable = true)
 |-- TaxiOut: integer (nullable = true)
 |-- Cancelled: integer (nullable = true)



In [12]:
old_amount = df.count()
df = df.na.drop("any")
new_amount = df.count()
print( "Before: " +str(old_amount) + ",\nAfter: " + str(new_amount) + ",\n%:"+str(round(new_amount/old_amount, 2)*100))

Before: 7129270,
After: 6987729,
%:98.0


In [13]:
df.show(5)

+-----+----------+---------+-------+----------+----------+-------------+---------+--------------+--------+--------+------+----+--------+-------+---------+
|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|CRSArrTime|UniqueCarrier|FlightNum|CRSElapsedTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiOut|Cancelled|
+-----+----------+---------+-------+----------+----------+-------------+---------+--------------+--------+--------+------+----+--------+-------+---------+
|    1|        12|        1|    623|       630|       915|           UA|      462|           105|     -14|      -7|   ORD| CLT|     599|     11|        0|
|    1|        13|        2|    621|       630|       915|           UA|      462|           105|      -4|      -9|   ORD| CLT|     599|     16|        0|
|    1|        14|        3|    633|       630|       915|           UA|      462|           105|       5|       3|   ORD| CLT|     599|     15|        0|
|    1|        15|        4|    627|       630|       915|           U

### LIN REG with only numerical values

In [14]:
corr_matrix = df.select([x[0] for x in df.dtypes if 'int' in x])

In [15]:
corr_matrix.show(5)

+-----+----------+---------+----------+----------+---------+--------------+--------+--------+--------+-------+---------+
|Month|DayofMonth|DayOfWeek|CRSDepTime|CRSArrTime|FlightNum|CRSElapsedTime|ArrDelay|DepDelay|Distance|TaxiOut|Cancelled|
+-----+----------+---------+----------+----------+---------+--------------+--------+--------+--------+-------+---------+
|    1|        12|        1|       630|       915|      462|           105|     -14|      -7|     599|     11|        0|
|    1|        13|        2|       630|       915|      462|           105|      -4|      -9|     599|     16|        0|
|    1|        14|        3|       630|       915|      462|           105|       5|       3|     599|     15|        0|
|    1|        15|        4|       630|       915|      462|           105|     -16|      -3|     599|     10|        0|
|    1|        16|        5|       630|       915|      462|           105|       3|       5|     599|     13|        0|
+-----+----------+---------+----

In [16]:
# I guess it is too pythonic and we nees to change it's PEARSON CORRELATION

[(c[0], df.corr("ArrDelay", c[0])) for c in corr_matrix.dtypes]

[('Month', 0.008202777323020096),
 ('DayofMonth', 0.024984890590987054),
 ('DayOfWeek', -0.017791803836727405),
 ('CRSDepTime', 0.1267987420931976),
 ('CRSArrTime', 0.12408222835628847),
 ('FlightNum', -0.001189494331136067),
 ('CRSElapsedTime', -0.013223108616871579),
 ('ArrDelay', 1.0),
 ('DepDelay', 0.907447881121435),
 ('Distance', -0.013181189620090234),
 ('TaxiOut', 0.25188155159775766),
 ('Cancelled', nan)]

In [17]:
features = df.select(['DepDelay', 'TaxiOut'])

In [18]:
from pyspark.ml.feature import VectorAssembler

In [19]:
assembler = VectorAssembler(
    inputCols=features.columns,
    outputCol="features")

In [20]:
#df = df.fillna(0, subset=(['DepDelay', 'Cancelled', 'ArrDelay']))

In [21]:
output = assembler.transform(df).select('features','ArrDelay')

In [22]:
output.take(5)

[Row(features=DenseVector([-7.0, 11.0]), ArrDelay=-14),
 Row(features=DenseVector([-9.0, 16.0]), ArrDelay=-4),
 Row(features=DenseVector([3.0, 15.0]), ArrDelay=5),
 Row(features=DenseVector([-3.0, 10.0]), ArrDelay=-16),
 Row(features=DenseVector([5.0, 13.0]), ArrDelay=3)]

In [23]:
train,test = output.randomSplit([0.75, 0.25])

In [24]:
train.show(25)

+------------+--------+
|    features|ArrDelay|
+------------+--------+
|   (2,[],[])|     -23|
|   (2,[],[])|     -23|
|   (2,[],[])|     -21|
|   (2,[],[])|     -19|
|   (2,[],[])|     -18|
|   (2,[],[])|     -15|
|   (2,[],[])|     -13|
|   (2,[],[])|     -12|
|   (2,[],[])|     -12|
|   (2,[],[])|     -10|
|   (2,[],[])|      -3|
|   (2,[],[])|      -2|
|   (2,[],[])|      -2|
|   (2,[],[])|       5|
|   (2,[],[])|       7|
|[-70.0,68.0]|     -10|
|[-60.0,18.0]|      10|
|[-50.0,70.0]|       3|
|[-39.0,15.0]|     -44|
| [-37.0,8.0]|     -38|
| [-36.0,5.0]|     -49|
|[-36.0,39.0]|     -18|
| [-34.0,3.0]|     -33|
|[-33.0,36.0]|      -3|
| [-32.0,7.0]|     -48|
+------------+--------+
only showing top 25 rows



In [25]:
from pyspark.ml.regression import LinearRegression
lin_reg = LinearRegression(featuresCol = 'features', labelCol='ArrDelay')
linear_model = lin_reg.fit(train)

In [26]:
print("Coefficients: " + str(linear_model.coefficients))
print("\nIntercept: " + str(linear_model.intercept))

Coefficients: [0.9999208558158891,0.4340318736425392]

Intercept: -8.294755647165388


In [27]:
trainSummary = linear_model.summary
print("RMSE: %f" % trainSummary.rootMeanSquaredError)
print("\nr2: %f" % trainSummary.r2)

RMSE: 12.439089

r2: 0.860394


In [28]:
from  pyspark.sql.functions import abs
predictions = linear_model.transform(test)
x =((predictions['ArrDelay']-predictions['prediction'])/predictions['ArrDelay'])*100
predictions = predictions.withColumn('Accuracy',abs(x))
predictions.select("prediction","ArrDelay","Accuracy","features").show(10)

+------------------+--------+------------------+--------------+
|        prediction|ArrDelay|          Accuracy|      features|
+------------------+--------+------------------+--------------+
|-8.294755647165388|     -34| 75.60365986127827|     (2,[],[])|
|-8.294755647165388|     -25| 66.82097741133845|     (2,[],[])|
|-8.294755647165388|     -12|30.877036273621766|     (2,[],[])|
|-8.294755647165388|      -8|3.6844455895673534|     (2,[],[])|
|-8.294755647165388|      -2|314.73778235826944|     (2,[],[])|
|-8.294755647165388|      -2|314.73778235826944|     (2,[],[])|
|-8.294755647165388|       1| 929.4755647165389|     (2,[],[])|
|-8.294755647165388|      12|169.12296372637823|     (2,[],[])|
|-236.1063931166072|      -7|3272.9484730943886|  [-230.0,5.0]|
|-96.77670545154987|       5|2035.5341090309973|[-174.0,197.0]|
+------------------+--------+------------------+--------------+
only showing top 10 rows



In [29]:
from pyspark.ml.evaluation import RegressionEvaluator
pred_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="ArrDelay",metricName="r2")
print("R Squared (R2) on test data = %g" % pred_evaluator.evaluate(predictions))

R Squared (R2) on test data = 0.862328


### Dealing with categorical data

In [30]:
NON_corr_matrix = df.select([x[0] for x in df.dtypes if x[1] !='int'])
NON_corr_matrix.show(5)

+-------+-------------+------+----+
|DepTime|UniqueCarrier|Origin|Dest|
+-------+-------------+------+----+
|    623|           UA|   ORD| CLT|
|    621|           UA|   ORD| CLT|
|    633|           UA|   ORD| CLT|
|    627|           UA|   ORD| CLT|
|    635|           UA|   ORD| CLT|
+-------+-------------+------+----+
only showing top 5 rows



In [31]:
origin = NON_corr_matrix.select("Origin").distinct().collect()

In [32]:
origin[:5]

[Row(Origin='BGM'),
 Row(Origin='DLG'),
 Row(Origin='MSY'),
 Row(Origin='GEG'),
 Row(Origin='BUR')]

In [33]:
from pyspark.ml.feature import StringIndexer

df = sqlContext.createDataFrame(
    [(0, "a"), (1, "b"), (2, "c"), (3, "a"), (4, "a"), (5, "c")],
    ["id", "category"])
indexer = StringIndexer(inputCol="category", outputCol="categoryIndex")
indexed = indexer.fit(df).transform(df)
indexed.show()

+---+--------+-------------+
| id|category|categoryIndex|
+---+--------+-------------+
|  0|       a|          0.0|
|  1|       b|          2.0|
|  2|       c|          1.0|
|  3|       a|          0.0|
|  4|       a|          0.0|
|  5|       c|          1.0|
+---+--------+-------------+



### Correlation experiments

In [35]:
# First approach
# from pyspark.ml.linalg import Vectors
# from pyspark.ml.stat import Correlation

# data = [(Vectors.sparse(4, [(0, 1.0), (3, -2.0)]),),
#         (Vectors.dense([4.0, 5.0, 0.0, 3.0]),),
#         (Vectors.dense([6.0, 7.0, 0.0, 8.0]),),
#         (Vectors.sparse(4, [(0, 9.0), (3, 1.0)]),)]
# df = spark.createDataFrame(data, ["features"])

# r1 = Correlation.corr(df, "features").head()
# print("Pearson correlation matrix:\n" + str(r1[0]))

# r2 = Correlation.corr(df, "features", "spearman").head()
# print("Spearman correlation matrix:\n" + str(r2[0]))

In [ ]:
#Second Approach
# from pyspark.ml.stat import Correlation
# from pyspark.ml.feature import VectorAssembler

# # convert to vector column first
# vector_col = "corr_features"
# assembler = VectorAssembler(inputCols=df.columns, outputCol=vector_col)
# df_vector = assembler.transform(df).select(vector_col)

# # get correlation matrix
# matrix = Correlation.corr(df_vector, vector_col)

In [ ]:
# matrix.collect()[0]["pearson({})".format(vector_col)].values

### Visualization experiments

In [37]:
# in order to visualize data, it has to be transformed in Pandas
#unfortunately, our dataset is too large, therefore we only have to get a sample
# in this case we only get 25% of our data, with no replacement

df_Pandas_25 = df.sample(False, 0.25, 42).toPandas()


In [ ]:
#I will be using Altair for visualization, which accepts only 5000 max observations
#from here we can tell what airports have the longest trips

import altair as alt

alt.Chart(df_Pandas_25.sample(n=5000, random_state=1)).mark_point().encode(
    x='Origin',
    y='Distance',
    color='DayOfWeek',
)

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(x=df_Pandas_25.Origin,
    y=df_Pandas_25.Distance)

In [39]:
sc.stop()